In [1]:
import os
os.chdir('../')

In [2]:
import argparse
import json
from glob import glob
import logging
from pathlib import Path
from typing import Any, Dict, Literal, List, Tuple

import torch
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
import lightning.pytorch as pl
from lightning.pytorch.loggers.csv_logs import CSVLogger
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping

from tqdm.auto import tqdm

import sys
import os

from proteinclip import contrastive
from proteinclip import data_utils
from proteinclip import hparams

from proteinclip.ppi_data import load_ppi_data
from proteinclip.model_utils import ONNXModel

PPI_DATA_DIR = data_utils.DATA_DIR / "ppi"


/usr/local/pace-apps/manual/packages/anaconda3/2023.03/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train_pairs, valid_pairs, test_pairs = (
    load_ppi_data(split) for split in ("train", "valid", "test")
)

In [4]:
all_keys = set()
for p in train_pairs + valid_pairs + test_pairs:
    all_keys.add(p[0])
    all_keys.add(p[1])


In [5]:
full_embed_map = pd.read_parquet('text+ESM+protclip.parquet')

In [6]:
full_embed_map.columns

Index(['organism', 'organism_id', 'name', 'evidence', 'function', 'id',
       'textual_embedding', 'sequence', 'proteinclip_embed', 'ESM_embed'],
      dtype='object')

In [7]:
embed_map = {
    row['id']: torch.from_numpy(row['ESM_embed']).float().numpy()  # Cast to Float32
    for _, row in tqdm(full_embed_map.iterrows(), desc="Loading relevant embeddings")
    if row['id'] in all_keys
}


Loading relevant embeddings: 0it [00:00, ?it/s]/tmp/ipykernel_496077/1835126053.py:2: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:203.)
  row['id']: torch.from_numpy(row['ESM_embed']).float().numpy()  # Cast to Float32
Loading relevant embeddings: 10000it [00:00, 27261.60it/s]


In [8]:
embed_map = {k: v / np.linalg.norm(v) for k, v in embed_map.items()}


In [11]:
os.getcwd()

'/storage/ice1/6/2/savunuri3/cse7850-compbio-project/proteinclip'

In [17]:
import onnxruntime as ort

session = ort.InferenceSession("pc_project_1_10000.onnx", providers=["CUDAExecutionProvider"])  # or "CPUExecutionProvider"


2025-04-26 12:31:34.307631217 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 499715, index: 0, mask: {1, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2025-04-26 12:31:34.307648658 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 499716, index: 1, mask: {2, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2025-04-26 12:31:34.307689520 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 499717, index: 2, mask: {3, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2025-04-26 12:31:34.309289357 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 499748, index: 33, mask: {34, }, error code: 22 error msg: Invalid argument. Specify the num

2025-04-26 12:31:34.379693920 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 499727, index: 12, mask: {13, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2025-04-26 12:31:34.383402372 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 499742, index: 27, mask: {28, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2025-04-26 12:31:34.387513300 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 499763, index: 47, mask: {48, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2025-04-26 12:31:34.390297517 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 499718, index: 3, mask: {4, }, error code: 22 error msg: Invalid argument. Specify the

In [18]:
input_name = session.get_inputs()[0].name
input_shape = session.get_inputs()[0].shape
input_type = session.get_inputs()[0].type
print(f"Input name: {input_name}, shape: {input_shape}, type: {input_type}")


Input name: input, shape: ['batch_size', 1280], type: tensor(float)


In [19]:
def generate_embeds(mp,batch_size=32):
    clip_embeds = []
    ls = []
    keys = []
    for i in mp:
        keys.append(i)
        ls.append(mp[i])

    for i in tqdm(range(0, len(ls), batch_size)):
        batch_seqs = ls[i:i+batch_size]


        batch_clip_plus_plus_embs = session.run(None, {input_name: batch_seqs})[0]  


        clip_embeds.extend(batch_clip_plus_plus_embs)
    
    ans = {}
    for ind,i in enumerate(keys):
        ans[i] = clip_embeds[ind]

    return ans

In [20]:
embed_map = generate_embeds(embed_map)

100%|██████████| 171/171 [00:00<00:00, 1322.88it/s]


We do this twice for ProteinCLIP+ (first one ESM -> ProteinCLIP, second one ProteinCLIP -> ProteinCLIP+)

In [21]:
session = ort.InferenceSession("pc+_project_1_10000.onnx", providers=["CUDAExecutionProvider"])  # or "CPUExecutionProvider"
input_name = session.get_inputs()[0].name
input_shape = session.get_inputs()[0].shape
input_type = session.get_inputs()[0].type
print(f"Input name: {input_name}, shape: {input_shape}, type: {input_type}")

Input name: input, shape: ['batch_size', 128], type: tensor(float)


2025-04-26 12:31:52.129149258 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 500011, index: 0, mask: {1, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2025-04-26 12:31:52.129191735 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 500012, index: 1, mask: {2, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2025-04-26 12:31:52.129223893 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 500013, index: 2, mask: {3, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2025-04-26 12:31:52.134304029 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 500057, index: 46, mask: {47, }, error code: 22 error msg: Invalid argument. Specify the num

In [22]:
embed_map = generate_embeds(embed_map)

100%|██████████| 171/171 [00:00<00:00, 6342.14it/s]


In [24]:
embed_map['P20393'].shape

(128,)

In [25]:
    train_dset, valid_dset, test_dset = [
        data_utils.LabeledPairDataset(p, embed_map)
        for p in (train_pairs, valid_pairs, test_pairs)
    ]


In [26]:
    train_dl, valid_dl, test_dl = [
        DataLoader(
            d, batch_size=128, shuffle=(i == 0), num_workers=16
        )
        for i, d in enumerate([train_dset, valid_dset, test_dset])
    ]


/home/hice1/savunuri3/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [27]:
    pl.seed_everything(seed=6489)


[rank: 0] Seed set to 6489


6489

In [28]:
net = contrastive.MLPForClassification(
    init_dim=next(iter(train_dl))[0].shape[-1],
    dims=[128, 1],
    lr=1e-4,
    unit_norm=False,
)


In [29]:
ckpt_callback = ModelCheckpoint(
    dirpath=None,
    filename="{epoch}-{val_auprc:.4f}",
    monitor="val_auprc",
    mode="max",
    save_top_k=1,
    save_weights_only=True,
    auto_insert_metric_name=True,
)

early_stop_callback = EarlyStopping(
    monitor='val_auprc',         # metric name to monitor
    patience=50,                 # epochs with no improvement after which to stop
    mode='max',                 # 'min' if you want to minimize the metric
    verbose=True                # optional: prints a message when triggered
)


In [30]:
    trainer = pl.Trainer(
        max_epochs=10000,
        accelerator="cuda",
        enable_progress_bar=True,
        # logger=logger,
        callbacks=[ckpt_callback, early_stop_callback],
        # log_every_n_steps=1,
        deterministic=True,
        devices=1
    )


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/hice1/savunuri3/.local/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [31]:
trainer.fit(net, train_dataloaders=train_dl, val_dataloaders=valid_dl)

You are using a CUDA device ('NVIDIA H100 80GB HBM3') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type       | Params | Mode 
----------------------------------------------
0 | layers | Sequential | 33.3 K | train
----------------------------------------------
33.3 K    Trainable params
0         Non-trainable params
33.3 K    Total params
0.133     Total estimated model params size (MB)
5         Modules in train mode
0         Modules in eval mode
SLURM auto-requeueing enabled. Setting signal handlers.


/home/hice1/savunuri3/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 0: 100%|██████████| 284/284 [00:03<00:00, 91.22it/s, v_num=2509891, train_loss=0.772, val_loss=0.694, val_auroc=0.556, val_auprc=0.554]

Metric val_auprc improved. New best score: 0.554


Epoch 1:   0%|          | 0/284 [00:00<?, ?it/s, v_num=2509891, train_loss=0.772, val_loss=0.694, val_auroc=0.556, val_auprc=0.554]          

...

Epoch 194:   0%|          | 0/284 [00:00<?, ?it/s, v_num=2509891, train_loss=0.364, val_loss=0.781, val_auroc=0.583, val_auprc=0.603]           

/home/hice1/savunuri3/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 194: 100%|██████████| 284/284 [00:02<00:00, 104.36it/s, v_num=2509891, train_loss=0.410, val_loss=0.778, val_auroc=0.586, val_auprc=0.606]

Monitored metric val_auprc did not improve in the last 50 records. Best score: 0.612. Signaling Trainer to stop.


Epoch 194: 100%|██████████| 284/284 [00:02<00:00, 104.29it/s, v_num=2509891, train_loss=0.410, val_loss=0.778, val_auroc=0.586, val_auprc=0.606]
